In [46]:
import json
import numpy as np
import pandas as pd
import requests
import numpy
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

# Research Question: Find the top 10 players with the highest ratio between completed passes and attempted passes for La Liga

### Import the players dataset

In [47]:

players=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\players.json")
players
#Displaying maximum number
pd.set_option('display.max_columns', 14)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Import the event data for Germany

In [48]:
events_ger=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\events\\events_Germany.json")
pd.set_option('display.max_columns', 15)
events_ger.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],15231,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739,Pass,2446,1H,2.409746,85,179896442
1,8,Simple pass,[{'id': 1801}],14786,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739,Pass,2446,1H,2.506082,85,179896443
2,8,Simple pass,[{'id': 1801}],14803,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739,Pass,2446,1H,6.946706,85,179896444
3,8,Simple pass,[{'id': 1801}],14768,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739,Pass,2446,1H,10.786491,85,179896445
4,8,Simple pass,[{'id': 1801}],14803,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739,Pass,2446,1H,12.684514,85,179896446


#### As we have to calculate the efficiency of the players based on the passes, we filter out the information related to passes. Thus we target the column 'eventName' and choose only those rows which contains information related to passes. 

In [49]:
events_ger['eventName']=events_ger['eventName'].astype(str)
events_ger=events_ger[(events_ger.eventName.str.contains('Pass'))]
pd.set_option('display.max_columns', 15)
events_ger.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],15231,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739,Pass,2446,1H,2.409746,85,179896442
1,8,Simple pass,[{'id': 1801}],14786,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739,Pass,2446,1H,2.506082,85,179896443
2,8,Simple pass,[{'id': 1801}],14803,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739,Pass,2446,1H,6.946706,85,179896444
3,8,Simple pass,[{'id': 1801}],14768,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739,Pass,2446,1H,10.786491,85,179896445
4,8,Simple pass,[{'id': 1801}],14803,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739,Pass,2446,1H,12.684514,85,179896446


#### Splitting the column 'Tags' that is a dictionary

In [50]:
TAG1=[]
for i in range(len(events_ger.tags)):
    try:
        TAG1.append(str(events_ger.tags[i][0]['id']))
    except:
        TAG1.append('Notag')
TAG2=[]
for i in range(len(events_ger.tags)):
    try:
        TAG2.append(str(events_ger.tags[i][1]['id']))
    except:
        TAG2.append('Notag')
TAG3=[]
for i in range(len(events_ger.tags)):
    try:
        TAG3.append(str(events_ger.tags[i][2]['id']))
    except:
        TAG3.append('Notag')
TAG4=[]
for i in range(len(events_ger.tags)):
    try:
        TAG4.append(str(events_ger.tags[i][2]['id']))
    except:
        TAG4.append('Notag')
TAG1=pd.DataFrame({'Tag1':TAG1})
TAG2=pd.DataFrame({'Tag2':TAG2})
TAG3=pd.DataFrame({'Tag3':TAG3})
TAG4=pd.DataFrame({'Tag4':TAG4})
TAG=[TAG1,TAG2,TAG3,TAG4]
TAG=pd.concat(TAG ,axis=1, sort=False)
mix_events_ger=[events_ger,TAG]
events_ger=pd.concat(mix_events_ger,  axis=1,  sort=False).reindex(events_ger.index)
events_ger=events_ger.drop('tags',axis=1)
events_ger.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,15231.0,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739.0,Pass,2446.0,1H,2.409746,85,179896442.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,14786.0,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739.0,Pass,2446.0,1H,2.506082,85,179896443.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,14803.0,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739.0,Pass,2446.0,1H,6.946706,85,179896444.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,14768.0,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739.0,Pass,2446.0,1H,10.786491,85,179896445.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,14803.0,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739.0,Pass,2446.0,1H,12.684514,85,179896446.0,1801,Notag,Notag,Notag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519389,8.0,Simple pass,16843.0,"[{'y': 36, 'x': 32}, {'y': 62, 'x': 32}]",2517044.0,Pass,2463.0,2H,2804.460822,85,251206860.0,NaN,NaN,NaN,NaN
519390,8.0,Simple pass,275384.0,"[{'y': 62, 'x': 32}, {'y': 88, 'x': 66}]",2517044.0,Pass,2463.0,2H,2807.540961,85,251206861.0,NaN,NaN,NaN,NaN
519391,8.0,High pass,15194.0,"[{'y': 88, 'x': 66}, {'y': 51, 'x': 89}]",2517044.0,Pass,2463.0,2H,2810.604426,83,251206862.0,NaN,NaN,NaN,NaN
519392,8.0,Head pass,14917.0,"[{'y': 49, 'x': 11}, {'y': 50, 'x': 14}]",2517044.0,Pass,2451.0,2H,2812.356433,82,251206779.0,NaN,NaN,NaN,NaN


In [51]:
events_ger['eventName'] = events_ger['eventName'].astype(str)
events_ger=events_ger[(events_ger.eventName.str.contains('Pass'))]
events_ger.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,15231.0,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739.0,Pass,2446.0,1H,2.409746,85,179896442.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,14786.0,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739.0,Pass,2446.0,1H,2.506082,85,179896443.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,14803.0,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739.0,Pass,2446.0,1H,6.946706,85,179896444.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,14768.0,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739.0,Pass,2446.0,1H,10.786491,85,179896445.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,14803.0,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739.0,Pass,2446.0,1H,12.684514,85,179896446.0,1801,Notag,Notag,Notag


#### Counting the number of passes with respect to the player id

In [19]:
passes=events_ger
passes=passes.drop('positions',axis=1)
passes.playerId=list(passes.playerId)
PLAYERS=[]
for i in range(len(passes.playerId)):
    PLAYERS.append(passes.playerId.iloc[i])
passes_player_ger=pd.DataFrame(PLAYERS , columns=['PlayerId'])
passes_player_ger['Numberofpasses'] = passes_player_ger.groupby('PlayerId')['PlayerId'].transform('count')
passes_player_ger

,PlayerId,Numberofpasses
0,15231.0,716
1,14786.0,509
2,14803.0,1473
3,14768.0,883
4,14803.0,1473
...,...,...
261457,16843.0,1444
261458,275384.0,920
261459,15194.0,522
261460,14917.0,1336


#### In order to set the threshold of the number of passes, we calculate the first quartile of the number of passes. Then we set the lower limit as mean-1.5$\times$sd. The reason behind this is, while dealing with a set of variables, we could give mean-2$\times$ sd. But the value of twice of sd exceeds the mean. So, we go for mean-1.5$\times$sd. Here we are concerned about the players who have played very limited number of matches or insufficient amount of hours. That is why we set the lower limit as the only threshold.

In [20]:
#calculating Mean
#Calculating Standard deviation
Mean=passes_player_ger.Numberofpasses.mean()
std=passes_player_ger.Numberofpasses.std()
Mean=int(Mean)
std=int(std)
#Calculating Lower bound
lower_bound=Mean-std
ID_passes_threshold_ger=[]
frequency_passes_threshold_ger=[]
frequency_passes=[]
ID_passes=[]
for i in range(len(passes_player_ger.Numberofpasses)):
    frequency_passes.append(float(passes_player_ger.Numberofpasses.iloc[i]))
    ID_passes.append(float(passes_player_ger.PlayerId.iloc[i]))
for i in range(len(frequency_passes)):
    if frequency_passes[i]>lower_bound:
        frequency_passes_threshold_ger.append(int(frequency_passes[i]))
        ID_passes_threshold_ger.append(int(ID_passes[i]))

In [21]:
passes_threshold_ger=pd.DataFrame(ID_passes_threshold_ger, columns=['PlayerId'])
passes_threshold_ger.insert(1, "Numberofpasses",frequency_passes_threshold_ger, True)
passes_threshold_ger.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
passes_threshold_ger

,PlayerId,Numberofpasses
0,15231,716
1,14786,509
2,14803,1473
3,14768,883
5,40657,1163
...,...,...
78219,221172,592
103200,14887,584
110615,19720,742
128400,297957,580


In [52]:
events_ger_tag2=events_ger.loc[events_ger['Tag2'] == '1801']
events_ger_tag1=events_ger.loc[events_ger['Tag1']=='1801']
events_ger_tag3=events_ger.loc[events_ger['Tag3']=='1801']
events_ger_tag4=events_ger.loc[events_ger['Tag4']=='1801']
events_ger_tag=[events_ger_tag1,events_ger_tag2,events_ger_tag3,events_ger_tag4]
success_ger=pd.concat(events_ger_tag, axis=0)
success_ger.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,15231.0,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739.0,Pass,2446.0,1H,2.409746,85,179896442.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,14786.0,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739.0,Pass,2446.0,1H,2.506082,85,179896443.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,14803.0,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739.0,Pass,2446.0,1H,6.946706,85,179896444.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,14768.0,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739.0,Pass,2446.0,1H,10.786491,85,179896445.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,14803.0,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739.0,Pass,2446.0,1H,12.684514,85,179896446.0,1801,Notag,Notag,Notag


In [24]:
S=[]
for i in range(len(success_ger.playerId)):
   S.append(success_ger.playerId.iloc[i])
success_ger_new=pd.DataFrame(S , columns=['PlayerId'])
success_ger_new['Numberofpasses'] = success_ger_new.groupby('PlayerId')['PlayerId'].transform('count')
success_ger_new.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
success_ger_new

,PlayerId,Numberofpasses
0,15231.0,263
1,14786.0,237
2,14803.0,629
3,14768.0,393
5,40657.0,459
...,...,...
94854,14887.0,77
96732,372316.0,1
98961,399599.0,9
99320,217052.0,8


In [36]:
final_passes_ger=pd.merge(success_ger_new, passes_threshold_ger, on='PlayerId')
final_passes_ger=final_passes_ger.rename(columns={'Numberofpasses_x':'AccuratePassesNumber','Numberofpasses_y':'TotalPassesNumber'})
final_passes_ger

,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,15231.0,263,716
1,14786.0,237,509
2,14803.0,629,1473
3,14768.0,393,883
4,40657.0,459,1163
...,...,...,...
221,134392.0,311,956
222,14988.0,155,666
223,14789.0,79,506
224,221172.0,71,592


In [40]:
def make_col(df, col):
    return pd.concat([df.drop([col], axis=1), df[col].apply(pd.Series)], axis=1)
teams = pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\teams.json").rename(columns={'name':'teamname'})
teams = make_col(teams, "area")
teams = teams.drop(columns='city')
teams = teams.rename(columns={'name':'nation'})
gerteams = teams[(teams["alpha3code"]=="DEU") & (teams["type"]=="club")]
gerplayer = players.merge(gerteams, left_on="currentTeamId", right_on="wyId")
gerplayer=gerplayer.drop(columns=['birthArea','height','role','currentNationalTeamId','officialName','type','middleName','lastName','passportArea','weight','currentTeamId','firstName','foot'],axis=1)

gerplayer=gerplayer.drop(columns=['alpha3code','alpha2code','id','nation','wyId_y','teamname'])
gerplayer=gerplayer.rename(columns={"wyId_x": "PlayerId"})

final_player_passes_ger=pd.merge(gerplayer, final_passes_ger, on='PlayerId')
final_player_passes_ger


,birthDate,PlayerId,shortName,AccuratePassesNumber,TotalPassesNumber
0,1999-05-25,393247,I. Konat\u00e9,284,606
1,1994-03-17,65596,M. Sabitzer,382,829
2,1995-05-14,296341,Bernardo,235,682
3,1994-10-24,70975,Bruma,319,785
4,1997-05-27,282804,K. Laimer,207,604
...,...,...,...,...,...
215,1991-03-14,14749,G. Sakai,391,1130
216,1992-02-23,14857,K. Papadopoulos,473,1058
217,1990-09-18,14864,L. Holtby,98,524
218,1992-11-01,127796,F. Kosti\u0107,175,704


In [41]:
Efficiency_ger=[]
for i in range(len(final_player_passes_ger.AccuratePassesNumber)):
    total=final_player_passes_ger.TotalPassesNumber.iloc[i]
    accurate=final_player_passes_ger.AccuratePassesNumber.iloc[i]
    Ratio=accurate/total
    Efficiency_ger.append(Ratio)
Efficiency_ger=pd.DataFrame(Efficiency_ger)

In [42]:
final_player_passes_ger.insert(4,'Efficiency',Efficiency_ger)
final_player_ratio_ger=final_player_passes_ger
final_player_ratio_ger

,birthDate,PlayerId,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1999-05-25,393247,I. Konat\u00e9,284,0.468647,606
1,1994-03-17,65596,M. Sabitzer,382,0.460796,829
2,1995-05-14,296341,Bernardo,235,0.344575,682
3,1994-10-24,70975,Bruma,319,0.406369,785
4,1997-05-27,282804,K. Laimer,207,0.342715,604
...,...,...,...,...,...,...
215,1991-03-14,14749,G. Sakai,391,0.346018,1130
216,1992-02-23,14857,K. Papadopoulos,473,0.447070,1058
217,1990-09-18,14864,L. Holtby,98,0.187023,524
218,1992-11-01,127796,F. Kosti\u0107,175,0.248580,704


In [43]:
final_player_ratio_ger.to_json(r'C:\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw2\final_player_ratio_ger.json')

In [44]:
bestefficiency_ger=final_player_ratio_ger.nlargest(10, ['Efficiency']) 
bestefficiency_ger=bestefficiency_ger.reset_index()
bestefficiency_ger=bestefficiency_ger.drop(columns=['index','PlayerId'])
bestefficiency_ger=bestefficiency_ger.set_value(9, 'shortName', 'V.Lindelof')

bestefficiency_ger

C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,birthDate,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1991-09-27,M. Halstenberg,456,0.807080,565
1,1983-05-28,M. Lehmann,613,0.780892,785
2,1996-06-13,K. Coman,418,0.716981,583
3,1993-07-03,K. Demirbay,435,0.698234,623
4,1994-03-13,Y. Gerhardt,376,0.693727,542
5,1997-09-09,F. Uduokhai,581,0.676368,859
6,1988-01-15,S. Langkamp,468,0.642857,728
7,1988-09-05,N. \u015eahin,643,0.634748,1013
8,1995-09-18,M. Meyer,637,0.626969,1016
9,1987-05-22,V.Lindelof,687,0.619477,1109


### The above table shows the top 10 most efficient players. We can observe that the highest efficiency is 80.70% in the premier league dataset.